# Running and Observing Agents

In [7]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage

from dotenv import load_dotenv
import os

load_dotenv
api_key = os.getenv("OPENROUTER_API_KEY")
if not api_key:
    raise ValueError("API key is not set.")

model_client = OpenAIChatCompletionClient(
    base_url="https://openrouter.ai/api/v1",
    model = "google/gemini-2.0-flash-exp:free",
    api_key=api_key,
    model_info={
        "family":"google",
        "vision" : True,
        "function_calling":True,
        "json_output":False,
    }
)

x:\Autogen\autogen-env\Lib\site-packages\autogen_ext\models\openai\_openai_client.py:453: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(self._model_info)


In [8]:
async def web_search(query: str) -> str:
    """Find information on the web"""
    return "The Labrador Retriever or simply Labrador is a British breed of retriever gun dog."

In [9]:
agent = AssistantAgent(
    name = "assistant",
    model_client=model_client,
    tools = [web_search],
    system_message="Use tools to solve tasks"
)

In [10]:
result = await agent.run(task = "find the information about Labrador Retrieve")
print(result.messages[-1].content)

The Labrador Retriever or simply Labrador is a British breed of retriever gun dog.


# on_message() method

it is used to see inside a agent.what happening inside the agent.

cancellation token - iska mtkb hai ki hm bol rhe hai Stop whatever you are doing — user ne cancel kar diya hai!. function ko forcibly nahi rok rahe, balki usse politely keh rahe.

inner_message - it is produced by agent , they can be AgentEvenet, ye agent ki internal steps ko dikhata hai

chat__message - ye agent ke final message ko dikhata hai

In [17]:
from autogen_core import CancellationToken

async def assistant_run() -> None:
    response = await agent.on_messages(
        messages=[TextMessage(content = 'find the information about Labrador Retrieve', source='User')],
        cancellation_token=CancellationToken()
    )
    print(response.inner_messages)
    print('\n\n\n\n')
    print(response.chat_message)

In [19]:
await assistant_run()

[ToolCallRequestEvent(id='f81af4f4-ef89-4a01-841f-df938ebfce7e', source='assistant', models_usage=RequestUsage(prompt_tokens=82, completion_tokens=7), metadata={}, created_at=datetime.datetime(2025, 10, 5, 14, 20, 51, 892318, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='tool_0_web_search_acDRSUKd8vEYqahzTlAm', arguments='{"query":"Labrador Retriever"}', name='web_search')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='5f4bbcfe-9c7c-4626-a236-2f0016771ba6', source='assistant', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 10, 5, 14, 20, 51, 892318, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='The Labrador Retriever or simply Labrador is a British breed of retriever gun dog.', name='web_search', call_id='tool_0_web_search_acDRSUKd8vEYqahzTlAm', is_error=False)], type='ToolCallExecutionEvent')]





id='8d05a15b-f96b-4845-a6d7-556432009ecb' source='assistant' models_usage=None metadata={} created_at=datetime.datetime

# Streaming Messages

using on_messages_stream() - ye on_stream se better understnad krta hai ki agent kis cheez se hoke jaa rha h

In [20]:
from autogen_agentchat.ui import Console  # for formatting/priniting agent messages

async def assistant_run_stream() -> None:
    await Console(
        agent.on_messages_stream(
            messages=[TextMessage(content = 'find the information about Labrador Retrieve', source='User')],
            cancellation_token=CancellationToken()
        ),
        output_stats=True  # Enable stats printing
    ) 
    
await assistant_run_stream()


---------- ToolCallRequestEvent (assistant) ----------
[FunctionCall(id='tool_0_web_search_CrmmQemzQVa8WosCuvEE', arguments='{"query":"Labrador Retriever"}', name='web_search')]
[Prompt tokens: 131, Completion tokens: 7]
---------- ToolCallExecutionEvent (assistant) ----------
[FunctionExecutionResult(content='The Labrador Retriever or simply Labrador is a British breed of retriever gun dog.', name='web_search', call_id='tool_0_web_search_CrmmQemzQVa8WosCuvEE', is_error=False)]
---------- assistant ----------
The Labrador Retriever or simply Labrador is a British breed of retriever gun dog.
---------- Summary ----------
Number of inner messages: 2
Total prompt tokens: 131
Total completion tokens: 7
Duration: 7.04 seconds
